In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Some extra libraries we need
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
#from qiskit.circuit.library import ry
from numpy import pi, random, cos
from qiskit import QuantumRegister, ClassicalRegister


Traceback (most recent call last):
  Cell In[1], line 3
    from qiskit.tools.jupyter import *
ModuleNotFoundError: No module named 'qiskit.tools'

Use %tb to get the full traceback.


### How to measure a state regarding an (orthonormal) basis {|u>, |v>}
- We need to construct the unitary matrix U=(|u>,|v>)
- Then, we apply the *inverse* of U to the state we want to measure
- We measure with respect to {|0>,|1>}
- Finally, we apply U to the collapsed state

- If the result is 0, the final will have collapsed to |u>, so we have measured "u"
- If the result is 1, the state will have collapsed to |v>, so we have measured "v"

If we want to measure a qubit wrt the basis {|+>,|->}, we use H as the unitary matrix.

Remember that H is its own inverse!

In [ ]:
### measuring in the basis {|+>,|->}
measureH = QuantumCircuit(1,1)
measureH.h(0)
measureH.measure(0,0)
measureH.h(0)

measureH.draw()


In [ ]:
simulator = AerSimulator()
compiled_circuit = transpile(measureH, simulator)
job = simulator.run(compiled_circuit, shots=1000)
result = job.result()
counts = result.get_counts(measureH)

plot_histogram(counts)

We shall need to measure with respect to a basis which is the canonical basis, rotated $\pi/8$ or $\pi/8$.

How can we construct the corresponding unitary matrices?

Check this out and look for a qiskit quantum gate that can implement rotations...

https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html

In [ ]:
## implement a measure wrt the rotation of the canonical basis by pi/8
import matplotlib.pyplot as plt
import numpy as np
from math import pi
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, transpile
from qiskit.tools.visualization import circuit_drawer
from qiskit.quantum_info import state_fidelity
from qiskit import BasicAer

backend = BasicAer.get_backend('unitary_simulator')

aq = QuantumRegister(1,"aq")
ac = ClassicalRegister(1,"ac")

prt = QuantumCircuit(aq,ac)

prt.ry(-pi/4,aq)
prt.measure(aq,ac)
prt.ry(pi/4,aq)

prt.draw()


Now you should be ready to try and implement the CHSH version of the Aspect experiment...

(the random x and y bits were a bit tricky to me, but maybe there is a simple way to generate a random bit by using a measure on certain state...)

In [ ]:
simulator2 = AerSimulator()
compiled_circuit2 = transpile(prt, simulator2)
job2 = simulator2.run(compiled_circuit2, shots=10000)
result2 = job2.result()
counts2 = result2.get_counts(prt)

plot_histogram(counts2)

In [ ]:
x = ClassicalRegister(1,"qx")
X = QuantumRegister(1,"cx")

RandomValue=QuantumCircuit(X,x)
RandomValue.h(X)
RandomValue.measure(X,x)
RandomValue.draw()

In [ ]:
simulator3 = AerSimulator()
compiled_circuit3 = transpile(RandomValue, simulator3)
job3 = simulator3.run(compiled_circuit3, shots=10000)
result3 = job3.result()
counts3 = result3.get_counts(RandomValue)

plot_histogram(counts3)

In [ ]:
A = QuantumRegister(1,"A")
B = QuantumRegister(1,"B")
x = QuantumRegister(1,"x")
y = QuantumRegister(1,"y")
a = ClassicalRegister(1,"a")
b = ClassicalRegister(1,"b")
cx = ClassicalRegister(1,"cx")
cy = ClassicalRegister(1,"cy")

RandomValue = QuantumCircuit(A,B,x,y,cx,cy,a,b)

RandomValue.h(A)
RandomValue.cnot(A,B)
RandomValue.barrier()
RandomValue.h(x)
RandomValue.h(y)
RandomValue.barrier()

with RandomValue.if_test((a, 1)):
    RandomValue.ry(pi/4,A)

with RandomValue.if_test((b, 0)):
    RandomValue.ry(pi/4,B)
    
with RandomValue.if_test((b, 1)):
    RandomValue.ry(-pi/4,B)
    
RandomValue.barrier()

RandomValue.measure(A,a)
RandomValue.measure(B,b)
RandomValue.measure(x,cx)
RandomValue.measure(y,cy)
    

    
RandomValue.draw()
                    

In [ ]:
simulator4 = AerSimulator()
compiled_circuit4 = transpile(RandomValue, simulator4)
job4 = simulator4.run(compiled_circuit4, shots=200000)
result4 = job4.result()
counts4 = result4.get_counts(RandomValue)

plot_histogram(counts4)